In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [24]:
df = pd.read_csv(r'C:\Users\Gaspar\Desktop\Coctails_db\all_drinks.csv', delimiter = ",")

In [25]:
df_temp = pd.DataFrame()

In [26]:
df_temp = df[['strDrink', 'strInstructions']]

In [27]:
df_temp = df_temp.rename(columns={'strDrink': 'Cocktail', 'strInstructions': 'Instruction'})

In [28]:
ids = range(1, len(df_temp) + 1)

In [30]:
data_dict = {'ID': ids, 'Cocktails': df_temp['Cocktail'], 'Instruction': df_temp['Instruction']}
df_cocktail = pd.DataFrame(data_dict)

In [31]:
df_cocktail.head()

,ID,Cocktails,Instruction
0,1,'57 Chevy with a White License Plate,1. Fill a rocks glass with ice 2.add white cre...
1,2,1-900-FUK-MEUP,Shake ingredients in a mixing tin filled with ...
2,3,110 in the shade,Drop shooter in glass. Fill with beer
3,4,151 Florida Bushwacker,Combine all ingredients. Blend until smooth. G...
4,5,155 Belmont,Blend with ice. Serve in a wine glass. Garnish...


In [32]:
conn = sqlite3.connect('cocktails_database.db')

In [36]:
df_ingredients.to_sql('cocktail', conn, index=False)

546

In [37]:
conn.close()